# Flexibility in a Search Model 

## Model

- homogeneous workers with linear utility over wage $w(x;k)$ if employed and flow (dis)utility $b$ if unemployed
- heterogeneous firms endowed with flexibility level $k \in K = \{1,2,3\}$ costing $c(k)$ with linear profit $y(x;k)-w(x;k)-c(k)$
    - current assumption: $y(x;k) = kx$
- search parameters: discount rate $\rho$, unemployed meet firms at rate $\lambda$ (no on-the-job search), upon meeting draw match-specific productivity $x \sim G(x)$, bargaining parameter $\alpha$, employed face separation shock $\eta$


## Data Requirements 
- employed workers earn wage $w_i$ at firm with flexibility level $k$
- unemployed workers have unemployment durations of $t_i$

In [6]:
import numpy as np
import pandas as pd 
import scipy.stats as stats


In [3]:
df=pd.read_stata('workfile.dta', columns=['sex','employed', 'flex_sched_score', 'hrwage', 'dur']) #986,337 obs from 1984-2017; 711,205 for 2000-2017

In [5]:
df['flex_sched_score'].groupby(df['sex']).value_counts(normalize=True)

sex     flex_sched_score
male    1.0                 0.535855
        0.0                 0.289992
        2.0                 0.174153
female  1.0                 0.435028
        0.0                 0.415658
        2.0                 0.149314
Name: flex_sched_score, dtype: float64

In [14]:
def lognormpdf(x: np.array, μ: float, σ: float):
    """
    Calculates lognormal pdf without stats packages
    """
    
    denom = x * σ * np.sqrt(2*np.pi)
    exp_num = -(np.log(x)-μ)**2
    exp_denom = 2 * σ * σ
    num = np.exp(exp_num/exp_denom)
    
    return num/denom

In [15]:
def lognormsf(x: np.array, μ: float, σ: float):
    """
    Calculated lognormal cdf with scipy.stats normal cdf
    """
    
    lnx = np.log(x)
    num = lnx - μ
    denom = σ
    
    return 1-stats.norm.cdf(num/denom)

In [ ]:
def p_wage(wage: np.array, α: float, k: np.array, res_wage: np.array, c_k: np.array, μ: float, σ: float):
    """
    ** NOT GOING TO WORK BECAUSE CURRENTLY WAGE AND K NOT PAIRED **

    Calculates unconditional probability of a wage draw
    """
    
    g = (1/α*k) * lognormpdf( ( 1/α*k )*( wage - (1-α)*res_wage + α*c_k ), μ, σ )
    
    return g

In [ ]:
def p_match(k: np.array, res_wage: np.array, c_k: np.array, μ: float, σ: float):
    """
    Calculates the probability of a match 
    """
    
    G_tilde = lognormsf( ( 1/k )*( res_wage - c_k), μ, σ )

In [ ]:
def 